# Script to format the .sql file containing data to be MySQL friendly

## Read the content of the file containing the database data

In [1]:
with open('../original_data/raw_data_from_MS_SQL_Server.sql', 'r') as file:
    content = file.readlines()

In [2]:
content[:10]

['USE [Sale_2019]\n',
 'GO\n',
 "INSERT [dbo].[Sales_April_2019] ([Order_ID], [Product], [Quantity_Ordered], [Price_Each], [Order_Date], [Purchase_Address]) VALUES (176558, N'USB-C Charging Cable', 2, 11.949999809265137, CAST(N'2019-04-19T08:46:00.0000000' AS DateTime2), N'917 1st St, Dallas, TX 75001')\n",
 'GO\n',
 'INSERT [dbo].[Sales_April_2019] ([Order_ID], [Product], [Quantity_Ordered], [Price_Each], [Order_Date], [Purchase_Address]) VALUES (NULL, NULL, NULL, NULL, NULL, NULL)\n',
 'GO\n',
 "INSERT [dbo].[Sales_April_2019] ([Order_ID], [Product], [Quantity_Ordered], [Price_Each], [Order_Date], [Purchase_Address]) VALUES (176559, N'Bose SoundSport Headphones', 1, 99.989997863769531, CAST(N'2007-04-19T22:30:00.0000000' AS DateTime2), N'682 Chestnut St, Boston, MA 02215')\n",
 'GO\n',
 "INSERT [dbo].[Sales_April_2019] ([Order_ID], [Product], [Quantity_Ordered], [Price_Each], [Order_Date], [Purchase_Address]) VALUES (176560, N'Google Phone', 1, 600, CAST(N'2012-04-19T14:38:00.0000000

## Some modifications to the content of the file

### Add a comma to every line

In [3]:
content = [i.replace('\n', ',\n') for i in content]

### Remove the square brackets:

In [4]:
content = [i.replace('[', '').replace(']', '') for i in content]

### Remove the `GO` line used in T-SQL

In [5]:
content = [i for i in content if i != 'GO,\n']

### Remove the string: `CAST(N'timestamp' AS DateTime2), N` by just the `timestamp` value. Also remove `N'` from varchar values.

In [6]:
content = [i.replace('CAST(N', '').replace(' AS DateTime2), N', ', ').replace("N'", "'") for i in content]

### Remove the schema name prefix and table names:

In [7]:
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
table_names = [f'dbo.Sales_{i.title()}_2019' for i in months]

In [8]:
# February has a slightly different table name:
table_names[1] = 'dbo.sales_feb_2019'

In [9]:
for table in table_names:
    content = [i.replace(table, '') for i in content]

Many records from February sales have null Order_ID values. They have been added to the data file as empty string (`''`). They must be replaced by `null`.

In [23]:
for index, line in enumerate(content):
    if line.startswith("('',"):
        content[index] = line.replace("(''", "(NULL")

### Change first line to: `USE sales2019':

In [12]:
content[0] = 'USE sales2019 ;\n'

### Add keyword `INTO` and database name `sales2019` after `INSERT`:

In [13]:
content[1] = content[1].replace('INSERT ', 'INSERT INTO sales2019')

### Remove the Insert statement from every line:

In [14]:
content[2:] = [i.replace('INSERT  (Order_ID, Product, Quantity_Ordered, Price_Each, Order_Date, Purchase_Address) VALUES ', '') for i in content[2:] ]

### Personally, I like to have lower case column names, so I will replace the names of the columns in the insert statement:

In [15]:
lowercase_column_names = 'Order_ID, Product, Quantity_Ordered, Price_Each, Order_Date, Purchase_Address'.lower()
content[1] = content[1].replace('Order_ID, Product, Quantity_Ordered, Price_Each, Order_Date, Purchase_Address', lowercase_column_names)

### Finally, add a semicolon to last line:

In [16]:
content[-1] = content[-1].replace(',\n', ';')

## Store the resulting lines in a new processed file

In [24]:
with open('data_processed.sql', 'w') as file:
    file.writelines(content)